In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under theinput directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


# LB Competition:

## 1. EDA

In [2]:
# ## Loading The Data

# # Change to full Dataseet  for Massive Training and for testing
# df_train = pd.read_csv('/kaggle/input/leash-BELKA/train.csv', nrows=1000)
# df_test = pd.read_csv('/kaggle/input/leash-BELKA/test.csv', nrows=1000)


In [3]:
# df_train.head()

In [4]:
# !pip install rdkit

### Plotting 2d Image

In [5]:
# import pandas as pd
# from rdkit import Chem
# from rdkit.Chem import Draw

# def plot_2d_molecule(df, molecule_id):
#     # Filter the DataFrame for the given id
#     row = df[df['id'] == molecule_id]
#     # Get the SMILES string for the molecule
#     smiles_string = row['molecule_smiles'].values[0]
#     # Generate an RDKit molecule object
#     mol = Chem.MolFromSmiles(smiles_string)
#     # Use RDKit to draw the molecule
#     img = Draw.MolToImage(mol)
#     # Display the image
#     return img



In [6]:
# #Example usage:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# img = plot_2d_molecule(df_train, 1)
# plt.imshow(img)
# plt.show()

## Plotting 3d Image

In [7]:
# !pip install py3Dmol

In [8]:

# from rdkit.Chem import AllChem
# import py3Dmol

# def plot_3d_molecule_by_id(df, molecule_id):
#     # Filter the DataFrame for the given id
#     row = df[df['id'] == molecule_id]
#     if row.empty:
#         return "No entry with the given ID."

#     # Get the SMILES string for the molecule
#     smiles_string = row['molecule_smiles'].values[0]
#     if not smiles_string:
#         return "No SMILES string found for the given ID."
    
#     # Create a molecule from a SMILES string
#     mol = Chem.MolFromSmiles(smiles_string)
#     if mol is None:
#         return "Invalid SMILES string."
    
#     # Generate 3D coordinates
#     mol = Chem.AddHs(mol)  # Add hydrogens
#     AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # Embed molecule in 3D space using ETKDG methodology
#     AllChem.UFFOptimizeMolecule(mol)  # Optimize the geometry

#     # Use py3Dmol for visualization
#     mb = Chem.MolToMolBlock(mol)
#     viewer = py3Dmol.view(width=800, height=400)
#     viewer.addModel(mb, "mol")
#     viewer.setStyle({'stick': {}})
#     viewer.zoomTo()
#     return viewer.show()



In [9]:
# # Example usage:
# plot_3d_molecule_by_id(df_train, 1)


In [10]:
# !pip install rdkit-pypi networkx matplotlib


In [11]:

# import pandas as pd
# import networkx as nx
# import matplotlib.pyplot as plt
# from rdkit import Chem
# from rdkit.Chem import Draw
# from rdkit.Chem.rdmolops import GetAdjacencyMatrix

# def plot_molecule_graph(df, molecule_id):
#     # Find the molecule by ID
#     row = df[df['id'] == molecule_id]
#     if row.empty:
#         print("No entry with the given ID.")
#         return

#     # Extract the SMILES string
#     smiles = row['molecule_smiles'].values[0]
#     mol = Chem.MolFromSmiles(smiles)
#     if mol is None:
#         print("Invalid SMILES string.")
#         return

#     # Create a graph from the molecule
#     G = nx.Graph(Chem.rdmolops.GetAdjacencyMatrix(mol))
#     labels = {}
#     for idx, atom in enumerate(mol.GetAtoms()):
#         G.nodes[idx]['label'] = atom.GetSymbol()
#         labels[idx] = atom.GetSymbol()

#     # Draw the graph
#     pos = nx.spring_layout(G)  # positions for all nodes
#     nx.draw(G, pos, with_labels=True, labels=labels, node_size=700, node_color='skyblue', font_size=16, font_weight='bold', edge_color='gray')
#     plt.title(f'Molecular graph of ID {molecule_id}')
#     plt.show()




In [12]:
# # Example usage:
# plot_molecule_graph(df_train, 1)

# Model 1 - SMILESBerth


In [13]:
import pandas as pd

# Define file path
csv_file = '/kaggle/input/leash-BELKA/train.csv'

# Load first 10M rows
train_data = pd.read_csv(csv_file, nrows=50000000)

# Filter for ones and zeros
df_ones = train_data[train_data['binds'] == 1]
df_zeros = train_data[train_data['binds'] == 0]

df_zeros_balanced = df_zeros.sample(n=len(df_ones))
df_train = pd.concat([df_ones, df_zeros_balanced]).sample(frac=1, random_state=42)

# Load the next 100K rows for evaluation
df_eval = pd.read_csv(csv_file, skiprows=range(1, 10000001), nrows=100000)

# Now you have your train and eval datasets ready
print("Training and evaluation datasets prepared.")


Training and evaluation datasets prepared.


In [14]:
df_eval = pd.read_csv(csv_file, skiprows=range(1, 10000001), nrows=100000)

In [15]:
del train_data

In [16]:
import gc
gc.collect()

13

In [17]:
# # Save datasets if necessary
# df_train_balanced.to_csv('/path/to/save/train_balanced.csv', index=False)
# df_eval.to_csv('/path/to/save/eval.csv', index=False)

# # Or proceed with further data processing, feature engineering, etc.


In [18]:
# from sklearn.model_selection import train_test_split

# # Split df_train to create a new training set and an evaluation set
# df_train, df_eval = train_test_split(df, test_size=0.1, random_state=42)


In [19]:
len(df_train)

476066

In [20]:
df_eval.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,10000000,C=CC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,COC(=O)c1cscc1N.Cl,Cc1cccc2sc(N)nc12,C=CC[C@H](Nc1nc(Nc2nc3c(C)cccc3s2)nc(Nc2cscc2C...,HSA,0
1,10000001,C=CC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,COC(=O)c1cscc1N.Cl,Cc1cccc2sc(N)nc12,C=CC[C@H](Nc1nc(Nc2nc3c(C)cccc3s2)nc(Nc2cscc2C...,sEH,0
2,10000002,C=CC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,COC(=O)c1cscc1N.Cl,Cc1ccnc(N)c1,C=CC[C@H](Nc1nc(Nc2cc(C)ccn2)nc(Nc2cscc2C(=O)O...,BRD4,0
3,10000003,C=CC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,COC(=O)c1cscc1N.Cl,Cc1ccnc(N)c1,C=CC[C@H](Nc1nc(Nc2cc(C)ccn2)nc(Nc2cscc2C(=O)O...,HSA,0
4,10000004,C=CC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,COC(=O)c1cscc1N.Cl,Cc1ccnc(N)c1,C=CC[C@H](Nc1nc(Nc2cc(C)ccn2)nc(Nc2cscc2C(=O)O...,sEH,0


In [21]:
!pip install transformers torch


In [22]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "seyonec/PubChem10M_SMILES_BPE_450k"
tokenizer = AutoTokenizer.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/101k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [23]:
from transformers import Trainer, TrainingArguments
from sklearn.preprocessing import OneHotEncoder


# Tokenizing the SMILES strings

train_encodings = tokenizer(df_train['molecule_smiles'].tolist(), truncation=True, padding=True, max_length=512)
eval_encodings = tokenizer(df_eval['molecule_smiles'].tolist(), truncation=True, padding=True, max_length=512)



2024-05-07 06:16:38.362139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 06:16:38.362268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 06:16:38.499150: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [24]:

# Encode protein names
encoder = OneHotEncoder(sparse=False)
train_proteins = encoder.fit_transform(df_train['protein_name'].values.reshape(-1, 1))
eval_proteins = encoder.transform(df_eval['protein_name'].values.reshape(-1, 1))


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [25]:

# Creating a Dataset object for Hugging Face
import torch

class SMILESDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, protein_encodings):
        self.encodings = encodings
        self.labels = labels
        self.protein_encodings = protein_encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        item['protein_encodings'] = torch.tensor(self.protein_encodings[idx], dtype=torch.float32)
        return item

    def __len__(self):
        return len(self.labels)





In [26]:
import torch.nn as nn

class ProteinBindingModel(nn.Module):
    def __init__(self, transformer_model, num_protein_features):
        super(ProteinBindingModel, self).__init__()
        self.transformer = transformer_model
        self.protein_features = nn.Linear(num_protein_features, 256)
        self.classifier = nn.Linear(768 + 256, 2)  # Adjust sizes accordingly if using a different model

    def forward(self, input_ids, attention_mask, protein_features):
        # Obtain the outputs from the transformer model
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        
        # Use the last hidden state
        last_hidden_state = outputs.last_hidden_state  # This is generally [batch size, sequence length, hidden size]
        pooled_output = last_hidden_state[:, 0, :]  # Take the first token's embeddings across all batches
        
        # Process protein features
        protein_features = self.protein_features(protein_features)
        
        # Concatenate pooled output and protein features
        combined_features = torch.cat((pooled_output, protein_features), dim=1)
        output = self.classifier(combined_features)
        return output



In [27]:
#create Dataset
train_dataset = SMILESDataset(train_encodings, df_train['binds'].tolist(), train_proteins)
eval_dataset = SMILESDataset(eval_encodings, df_eval['binds'].tolist(), eval_proteins)

In [28]:
# Load the base transformer model without the classification head
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).base_model
model = ProteinBindingModel(base_model, train_proteins.shape[1])

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/',           # output directory
    num_train_epochs=30,                      # number of training epochs
    per_device_train_batch_size=16,          # batch size for training
    per_device_eval_batch_size=64,           # batch size for evaluation
    warmup_steps=500,                        # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                       # strength of weight decay
    logging_dir='/kaggle/working/logs',      # directory for storing logs
    logging_steps=1000,
    report_to="none",                        # suppress logging to WANDB
    save_steps=10000,                        # save model every 10000 steps
    evaluation_strategy='steps',             # evaluate each `eval_steps`
    eval_steps=5000,                         # evaluation every 5000 steps
)


In [30]:

gc.collect()

46

In [31]:
class ProteinBindingTrainer(Trainer):
    def _prepare_inputs(self, inputs):
        # Instead of modifying inputs, just pass them through.
        return inputs

    def training_step(self, model, batch):
        # Move batch to device
        batch = {k: v.to(self.args.device) for k, v in batch.items() if hasattr(v, 'to')}

        # Extract inputs from batch
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        protein_features = batch['protein_encodings']

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, protein_features=protein_features)
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs[0]
        
        # Compute loss
        loss = self.compute_loss(logits, labels)

        return loss


In [32]:
test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)
test_batch = next(iter(test_loader))
print(test_batch.keys())  # This should include 'labels' and 'protein_encodings'


dict_keys(['input_ids', 'attention_mask', 'labels', 'protein_encodings'])


In [33]:
# # Ensure the model is on the correct device
# model.to(device)

#THIS TRAINER IS NOT WORKING... SOMEHOW THE FUNCTION IS NOT GETTNG THE KEYS PROPERLY

# # Initialize the trainer with all settings
# trainer = ProteinBindingTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,  # Ensure this is your dataset, not DataLoader
#     eval_dataset=eval_dataset     # Ensure this is your dataset, not DataLoader
# )

# # Start training
# trainer.train()



In [34]:
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)



Using device: cuda


In [35]:
import torch

# Real-world frequencies
real_world_freq_0 = 0.9946  # 99.46% are class 0
real_world_freq_1 = 0.0054  # 0.54% are class 1

# Calculate class weights
weights = torch.tensor([1 / real_world_freq_0, 1 / real_world_freq_1], dtype=torch.float32)
weights = weights / weights.sum() * 2  # Normalize so that weights sum to the number of classes (2 for binary classification)
weights = weights.to(device)  # Move weights to the correct device

# Initialize the loss function with these weights
criterion = torch.nn.CrossEntropyLoss(weight=weights)


In [36]:
from torch.optim import Adam
from torch.utils.data import DataLoader

# Assuming the model is already defined and moved to the correct device
model = model.to(device)  # Ensure model is on the correct device
optimizer = Adam(model.parameters(), lr=5e-5)  # Define optimizer

# Define DataLoader for both training and validation datasets
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=16)

print('Data Loaders Defined')

# Training loop
num_epochs = 3  # Define the number of epochs
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}  # Move batch data to the correct device

        # Forward pass
        outputs = model(input_ids=batch['input_ids'],
                        attention_mask=batch['attention_mask'],
                        protein_features=batch['protein_encodings'])

        # Compute loss using the criterion with class weights
        loss = criterion(outputs, batch['labels'])

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss every 100 batches
        if (i + 1) % 100 == 0:
            print(f"Epoch {epoch}, Batch {i + 1}, Loss: {loss.item()}")

    # Validation loop
    model.eval()
    total_loss = 0
    for batch in eval_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=batch['input_ids'],
                        attention_mask=batch['attention_mask'],
                        protein_features=batch['protein_encodings'])
        loss = criterion(outputs, batch['labels'])
        total_loss += loss.item()

    avg_loss = total_loss / len(eval_loader)
    print(f"Validation Loss: {avg_loss}")


Data Loaders Defined
Epoch 0, Batch 100, Loss: 0.02751481905579567
Epoch 0, Batch 200, Loss: 0.015931639820337296
Epoch 0, Batch 300, Loss: 0.06304077804088593
Epoch 0, Batch 400, Loss: 0.06091780215501785
Epoch 0, Batch 500, Loss: 0.030183887109160423
Epoch 0, Batch 600, Loss: 0.02811356447637081
Epoch 0, Batch 700, Loss: 0.010884014889597893
Epoch 0, Batch 800, Loss: 0.025038253515958786
Epoch 0, Batch 900, Loss: 0.03134114295244217
Epoch 0, Batch 1000, Loss: 0.0356045663356781
Epoch 0, Batch 1100, Loss: 0.03795817121863365
Epoch 0, Batch 1200, Loss: 0.05314461141824722
Epoch 0, Batch 1300, Loss: 0.41865110397338867
Epoch 0, Batch 1400, Loss: 0.04026712477207184
Epoch 0, Batch 1500, Loss: 0.014546114020049572
Epoch 0, Batch 1600, Loss: 0.012577733024954796
Epoch 0, Batch 1700, Loss: 0.011738534085452557
Epoch 0, Batch 1800, Loss: 0.013799854554235935
Epoch 0, Batch 1900, Loss: 0.028157053515315056
Epoch 0, Batch 2000, Loss: 0.04320543631911278
Epoch 0, Batch 2100, Loss: 0.01091883983

In [37]:

# from torch.utils.data import DataLoader
# from torch.optim import Adam
# import torch.nn.functional as F

# # Assuming the model, dataset, and device setup
# model = model.to(device)  # Ensure model is on the correct device
# optimizer = Adam(model.parameters(), lr=5e-5)  # Define optimizer

# # Define DataLoader for both training and validation datasets
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# eval_loader = DataLoader(eval_dataset, batch_size=16)

# # Training loop
# num_epochs = 3  # Define the number of epochs
# for epoch in range(num_epochs):
#     model.train()
#     for i, batch in enumerate(train_loader):
#         # Ensure all data in the batch is moved to the correct device
#         batch = {k: v.to(device) for k, v in batch.items()}

#         # Forward pass
#         outputs = model(input_ids=batch['input_ids'],
#                         attention_mask=batch['attention_mask'],
#                         protein_features=batch['protein_encodings'])
        
#         # Compute loss
#         loss = F.cross_entropy(outputs, batch['labels'])  # Assuming your model outputs raw logits

#         # Backward pass
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss every 50 batches
#         if (i + 1) % 50 == 0:
#             print(f"Epoch {epoch}, Batch {i + 1}, Loss: {loss.item()}")  # Output the loss

#     # Validation loop (optional, for model evaluation during training)
#     model.eval()
#     with torch.no_grad():
#         for batch in eval_loader:
#             batch = {k: v.to(device) for k, v in batch.items()}
#             outputs = model(input_ids=batch['input_ids'],
#                             attention_mask=batch['attention_mask'],
#                             protein_features=batch['protein_encodings'])
#             loss = F.cross_entropy(outputs, batch['labels'])
#             # Add additional code to track validation loss, accuracy, etc.


In [38]:
# Save only the state dict
torch.save(model.state_dict(), 'model_state_dict.pth')


In [39]:

# Save the entire model
torch.save(model, 'entire_model.pth')

## Testing

LOading and testing data

In [40]:
# df_test = pd.read_csv('/kaggle/input/leash-BELKA/test.csv')

In [41]:
# len(df_test)

In [42]:
# # Assuming df_test is your test DataFrame with similar columns as df_train except 'binds'
# test_encodings = tokenizer(df_test['molecule_smiles'].tolist(), truncation=True, padding=True, max_length=512)


In [43]:
# test_dataset = SMILESTestDataset(test_encodings)

In [44]:
# # Predict
# predictions = trainer.predict(test_dataset)

# # The predictions object includes a predictions array, label_ids array (if available), and metrics (if labels provided during prediction)
# pred_labels = predictions.predictions.argmax(-1)  # We use argmax to convert from logits to class labels


In [45]:
# df_test['predicted_binds'] = pred_labels

In [46]:
# # Saving the DataFrame with predictions to a new CSV file
# df_test.to_csv('test_predictions.csv', index=False)